In [156]:
import pandas as pd
import scipy.optimize as opt;

Df = pd.read_csv("../../membershipchainservice/Data/gentree-node_0-epoch8")
Df

,Name,Level,X,Y,cluster,bunch
0,node_0,2,136.319756,96.453369,map[node_1:true node_10:true node_11:true node...,map[node_10:true node_8:true]
1,node_1,0,132.080824,193.355114,map[node_17:true node_3:true],map[node_0:true node_10:true node_11:true node...
2,node_2,1,202.085381,283.017647,map[node_14:true node_16:true node_17:true nod...,map[node_0:true node_10:true node_8:true]
3,node_3,0,415.913550,158.737687,map[node_13:true node_16:true node_7:true],map[node_0:true node_1:true node_10:true node_...
4,node_4,1,159.368581,372.835810,map[node_12:true node_17:true node_3:true],map[node_0:true node_10:true node_11:true node...
5,node_5,0,399.999079,210.610503,map[node_14:true node_16:true],map[node_0:true node_10:true node_8:true]
6,node_6,0,471.606683,429.968657,map[node_13:true node_16:true],map[node_0:true node_10:true node_11:true node...
7,node_7,0,138.637300,29.277539,map[node_16:true node_3:true],map[node_0:true node_10:true node_15:true node...
8,node_8,2,389.320563,144.673657,map[node_0:true node_1:true node_10:true node_...,map[node_0:true node_10:true]
9,node_9,0,231.290577,377.867467,map[node_13:true],map[node_0:true node_10:true node_8:true]


In [157]:
cluster = Df["cluster"][1]
import re

regex = r"(node_[0-9]+)"
explode_list_lambda = lambda map_str : [m.group() for m in re.finditer(regex, map_str, re.MULTILINE)]
Df["cluster_list"] = Df["cluster"].apply(explode_list_lambda)
Df["bunch_list"] = Df["bunch"].apply(explode_list_lambda)
Df

,Name,Level,X,Y,cluster,bunch,cluster_list,bunch_list
0,node_0,2,136.319756,96.453369,map[node_1:true node_10:true node_11:true node...,map[node_10:true node_8:true],"[node_1, node_10, node_11, node_12, node_13, n...","[node_10, node_8]"
1,node_1,0,132.080824,193.355114,map[node_17:true node_3:true],map[node_0:true node_10:true node_11:true node...,"[node_17, node_3]","[node_0, node_10, node_11, node_12, node_15, n..."
2,node_2,1,202.085381,283.017647,map[node_14:true node_16:true node_17:true nod...,map[node_0:true node_10:true node_8:true],"[node_14, node_16, node_17, node_3, node_4]","[node_0, node_10, node_8]"
3,node_3,0,415.913550,158.737687,map[node_13:true node_16:true node_7:true],map[node_0:true node_1:true node_10:true node_...,"[node_13, node_16, node_7]","[node_0, node_1, node_10, node_11, node_12, no..."
4,node_4,1,159.368581,372.835810,map[node_12:true node_17:true node_3:true],map[node_0:true node_10:true node_11:true node...,"[node_12, node_17, node_3]","[node_0, node_10, node_11, node_12, node_15, n..."
5,node_5,0,399.999079,210.610503,map[node_14:true node_16:true],map[node_0:true node_10:true node_8:true],"[node_14, node_16]","[node_0, node_10, node_8]"
6,node_6,0,471.606683,429.968657,map[node_13:true node_16:true],map[node_0:true node_10:true node_11:true node...,"[node_13, node_16]","[node_0, node_10, node_11, node_12, node_8]"
7,node_7,0,138.637300,29.277539,map[node_16:true node_3:true],map[node_0:true node_10:true node_15:true node...,"[node_16, node_3]","[node_0, node_10, node_15, node_3, node_8]"
8,node_8,2,389.320563,144.673657,map[node_0:true node_1:true node_10:true node_...,map[node_0:true node_10:true],"[node_0, node_1, node_10, node_11, node_12, no...","[node_0, node_10]"
9,node_9,0,231.290577,377.867467,map[node_13:true],map[node_0:true node_10:true node_8:true],[node_13],"[node_0, node_10, node_8]"


In [158]:
def cover_cluster(radius, name, Df):
    node = Df[Df["Name"]==name]
    cl = list(Df[Df["Name"]==name].cluster_list)[0]
    if len(cl) == 0:
        return True
    
    distance_lambda = lambda row : np.sqrt((row.X-node.X)**2 + (row.Y-node.Y)**2)
    distances = Df[Df["Name"].isin(cl)].apply(distance_lambda, axis=1)
    if radius < distances.max().values[0]:
        return False
    
    return True
    
def compute_radius(name, Df, base):
    radius = base
    while not cover_cluster(radius, name, Df):
        radius *= 2
        
    print(name, radius)
    return radius

In [163]:
palette = [
    [64/255, 64/255, 122/255,1.0],
    [255/255, 121/255, 63/255,1.0],
    [52/255, 172/255, 224/255,1.0],
    [255/255, 177/255, 66/255,1.0],
]

In [164]:
from io import BytesIO
import numpy as np
import cairo
import IPython.display

svgio = BytesIO()
side = 1000

with cairo.SVGSurface(svgio, side, side) as surface:
    context = cairo.Context(surface)
    context.scale(0.3,0.3)
    context.translate(side/2, side/2)
    
    context.set_source_rgba(0,0,0,1)
    context.set_dash([1.0])
    context.set_line_width(0.5)
    for row in Df.iterrows():
        Name, Level, X, Y, cluster, bunch, cluster_list, bunch_list = row[1]
        context.arc(X, Y, compute_radius(Name, Df, 16), 0, 2*np.pi)
        context.stroke()
        
    
    
    
    for row in Df.iterrows():
        Name, Level, X, Y, cluster, bunch, cluster_list, bunch_list = row[1]
        context.set_source_rgba(**palette[Level])
        context.arc(X, Y, 8, 0, 2*np.pi)
        context.fill()
    
    context.set_source_rgba(1,1,1,1)
    for row in Df.iterrows():
        Name, Level, X, Y, cluster, bunch, cluster_list, bunch_list = row[1]
        context.move_to(X-2.5,Y+3.5)
        context.show_text(str(Level))
        context.stroke()
    
    
    
IPython.display.SVG(data=svgio.getvalue())

node_0 1024
node_1 512
node_2 256
node_3 512
node_4 512
node_5 512
node_6 512
node_7 512
node_8 512
node_9 512
node_10 512
node_11 1024
node_12 512
node_13 256
node_14 512
node_15 512
node_16 128
node_17 64


TypeError: set_source_rgba() argument after ** must be a mapping, not list

In [161]:
node = Df[Df["Name"]=="node_0"]
cl = list(Df[Df["Name"]=="node_0"].cluster_list)[0]

distance_lambda = lambda row : np.sqrt((row.X-node.X)**2 + (row.Y-node.Y)**2)
Df[Df["Name"].isin(cl)].apply(distance_lambda, axis=1)

,0
1,96.994416
2,197.816448
3,286.447248
4,277.341850
5,287.330187
6,472.916241
7,67.215795
8,257.555051
9,297.007325
10,264.290560
